In [1]:
import boto3
import os
import json
from dotenv import load_dotenv

In [2]:
# 認証情報を環境変数から読み込み
load_dotenv()


True

In [3]:
# 認証情報を直接指定
session = boto3.Session(
    aws_access_key_id=os.getenv('aws_access_key_id'),
    aws_secret_access_key=os.getenv('aws_secret_access_key'),
    region_name=os.getenv('region_name')
)



In [4]:
region_name = 'us-east-1'
# ナレッジベースID
knowledge_base_id = os.getenv("knowledge_base_id")
# データソースID
data_source_id = os.getenv("data_source_id")

kb_client_runtime = session.client("bedrock-agent-runtime", region_name=region_name)
bedlock_runtime = session.client(service_name="bedrock-runtime", region_name=region_name)

# 推論用モデル(今回はClaudeのv2を使います)
model_id = "anthropic.claude-v2"
# クエリ拡張用のモデル(Claude Instantを使います。これは簡単なタスクにおける速度向上が狙いです)
q_model_id = "anthropic.claude-instant-v1"

In [5]:
# 推論用コード
def invoke_claude(text, model_id, max_tokens_to_sample=1000):
    body = json.dumps({
        "prompt": f"\n\nHuman:{text}\n\nAssistant: ",
        "max_tokens_to_sample": max_tokens_to_sample,
        "temperature": 0.1,
        "top_p": 0.9,
    })
    accept = "application/json"
    content_type = "application/json"

    response = bedlock_runtime.invoke_model(
        body=body,
        modelId=model_id,
        accept=accept,
        contentType=content_type
    )

    response_body = json.loads(response.get('body').read())
    return response_body.get('completion')[1:]

# クエリ拡張用コード
def generate_queries(original_query, n=4):
    f = ''
    for i in range(n):
        f += f'{n}: \n'

    prompt = f'''
以下に示すQueryはユーザの入力した検索クエリです。
このクエリに関連するクエリを多角的な視点から{n}つ生成してください
Formatに従って結果を出力してください

<Query>
{original_query}
</Query>

<Format>
{f}
</Format>
'''
    result = invoke_claude(prompt, q_model_id)
    result = result.split('<Format>')[1].split('</Format>')[0]
    # print(result)
    generated_queries = []
    for q in result.split('\n'):
        if q == '':
            continue
        generated_queries.append(q.split(' ')[1])
    return generated_queries[1:-1]

In [6]:
# RAG検索用コード
def kb_search(query, n=5):
    res = kb_client_runtime.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=knowledge_base_id,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': n
            }
        }
    )

    # {doc: score}の形に整形します
    return_dict = {}
    for r in res['retrievalResults']:
        return_dict[r['content']['text']] = r['score']

    return return_dict

In [7]:
# RRFのコード
def reciprocal_rank_fusion(all_results, k=50):
    fused_scores = {}
    for query, doc_scores in all_results.items():
        for rank, (doc, score) in enumerate(sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)):
            if doc not in fused_scores:
                fused_scores[doc] = 0
            previous_score = fused_scores[doc]
            fused_scores[doc] += 1 / (rank + k)

    reranked_results = {doc: score for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)}
    return reranked_results

In [8]:
original_query = '確定申告について教えて'

queries = generate_queries(original_query)
queries.insert(0, original_query)

all_results = {}
for q in queries:
    all_results[q] = kb_search(q)

# ここでは上位5件を採用している
reranked_results = list(reciprocal_rank_fusion(all_results).keys())[:5]

information = ''
for i, r in enumerate(reranked_results):
    information += f'情報{i+1}. {r}\n'

prompt = """
日本では本日(2月16日)から確定申告の相談及び申告書の受付がスタートしましたね。
この国の風物詩であるこの確定申告という儀式では、毎年数多の迷える子羊が出現することで有名です。
あなたは全知全能の税神として、迷える子羊の抱いている確定申告に対する悩みを聞き、彼らを正しき納税へと導いてください。
全知全能たるあなたのために、Informationセクションには迷える子羊の悩みに関連しそうな情報を提供します。有効に活用してください。

<Information>
{information}
</Information>

<UserQuery>
{query}
</UserQuery>
"""
prompt = prompt.format(information=information, query=original_query)

print(invoke_claude(prompt, model_id))

はい、確定申告についてご質問いただきありがとうございます。

確定申告は、1年間の所得を正しく国に申告する大切な制度です。申告をすることで、所得に応じた正しい税金を納めることができます。

申告の際は、給与所得の源泉徴収票や不動産所得の証明書など、1年間の所得がわかる書類を準備する必要があります。医療費控除や住宅ローン控除など、控除を受けられる項目がある場合は関連する証拠書類も必要です。 

申告書の作成には注意が必要です。所得金額や控除項目を正確に記入し、税法に従って適切に計算することが大切です。分からない点があれば、税務署や税理士に相談することをおすすめします。

e-Taxなどの電子申告も便利なので活用することをおすすめします。申告期限は3月15日ですので、余裕をもって手続きを進めましょう。

このように、確定申告は国民の義務です。しっかりとした知識を持ち、正しく納税することが大切です。ご不明な点がありましたら遠慮なくお問い合わせください。
